In [5]:
#linear regression
import pyspark
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName('housing_price_model').getOrCreate()

df=spark.read.csv('./cruise_ship_info.csv',inferSchema=True,header=True)
df.show(10)


+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [6]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [7]:
df.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [8]:
from pyspark.ml.feature import StringIndexer
indexer=StringIndexer(inputCol='Cruise_line',outputCol='cruise_cat')
indexed=indexer.fit(df).transform(df)

for item in indexed.head(5):
    print(item)
    

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0)
Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0)
Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, cruise_cat=1.0)
Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1, cruise_cat=1.0)
Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0, cruise_cat=1.0)


In [10]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

assembler=VectorAssembler(inputCols=['Age', 
 'Tonnage', 
 'passengers', 
 'length', 
 'cabins', 
 'passenger_density', 
 'cruise_cat'],outputCol='features')
output=assembler.transform(indexed)
output.select('features','crew').show(5)


+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
+--------------------+----+
only showing top 5 rows



In [11]:
final_data=output.select('features','crew')
train_data,test_data=final_data.randomSplit([0.7,0.3])
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               123|
|   mean| 7.812520325203264|
| stddev|3.5593961528529197|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [13]:
from pyspark.ml.regression import LinearRegression

ship_lr=LinearRegression(featuresCol='features',labelCol='crew')
trained_ship_model=ship_lr.fit(train_data)
ship_results=trained_ship_model.evaluate(train_data)

print("Rsquared Error: ",ship_results.r2)


Rsquared Error:  0.9217017198307328


In [14]:
unlabeled_data=test_data.select('features')
unlabeled_data.show(5)

+--------------------+
|            features|
+--------------------+
|[5.0,122.0,28.5,1...|
|[5.0,133.5,39.59,...|
|[6.0,30.276999999...|
|[6.0,113.0,37.82,...|
|[8.0,91.0,22.44,9...|
+--------------------+
only showing top 5 rows



In [15]:
predictions=trained_ship_model.transform(unlabeled_data)
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[5.0,122.0,28.5,1...| 5.895647293863092|
|[5.0,133.5,39.59,...|12.999590152367317|
|[6.0,30.276999999...| 4.344935876595818|
|[6.0,113.0,37.82,...|11.571591896146687|
|[8.0,91.0,22.44,9...|10.058048893738766|
|[8.0,110.0,29.74,...| 12.12191854161595|
|[9.0,59.058,17.0,...| 7.613326119101266|
|[9.0,81.0,21.44,9...| 9.516152416991888|
|[9.0,90.09,25.01,...| 9.167861014569807|
|[9.0,110.0,29.74,...| 12.11076518672379|
|[9.0,116.0,26.0,9...|11.089381352592845|
|[10.0,46.0,7.0,6....|2.6258595477635307|
|[10.0,77.0,20.16,...|  8.75463458708946|
|[11.0,90.09,25.01...| 8.740995896888599|
|[11.0,108.977,26....|11.022998623576214|
|[12.0,42.0,14.8,7...|6.7513753261577065|
|[12.0,58.6,15.66,...|7.4010787225758286|
|[12.0,108.865,27....|10.758058397688334|
|[14.0,33.0,4.9,5....|2.9093122978154877|
|[14.0,76.8,19.6,8...| 8.849141095722189|
+--------------------+------------